In [13]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
tar: spark-3.3.1-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


Exception: ignored

In [14]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 21.7 kB in 1s (15.4 kB/s)
Reading package lists... Done


In [15]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [16]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=",", header=True)

In [26]:
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [27]:
# 2. Create a temporary view of the DataFrame.

home_sales_df.createOrReplaceTempView('homeSales')

In [31]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
start_time = time.time()
spark.sql("""
          SELECT ROUND(AVG(price), 2), 
          EXTRACT(year FROM date) AS year
          FROM homeSales GROUP BY EXTRACT(year FROM date)""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|year|
+--------------------+----+
|           314421.54|2022|
|           315782.18|2019|
|           315949.81|2020|
|           313218.09|2021|
+--------------------+----+

--- 0.9015851020812988 seconds ---


In [43]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("""
  SELECT ROUND(AVG(price), 2), EXTRACT(year FROM date_built) AS year
  FROM homeSales 
  WHERE bedrooms=3 AND bathrooms=3
  GROUP BY EXTRACT(year FROM date_built)
""").show()

+--------------------+----+
|round(avg(price), 2)|year|
+--------------------+----+
|            288770.3|2015|
|           295962.27|2013|
|           290852.27|2014|
|           293683.19|2012|
|           290555.07|2016|
|           292859.62|2010|
|           291117.47|2011|
|           292676.79|2017|
+--------------------+----+



In [44]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("""
SELECT ROUND(AVG(price), 2), EXTRACT(year FROM date_built) AS year
FROM homeSales
WHERE bedrooms=3 AND bathrooms=3 AND floors=2 AND sqft_living>=2000
GROUP BY EXTRACT(year FROM date_built)
""").show()

+--------------------+----+
|round(avg(price), 2)|year|
+--------------------+----+
|           297609.97|2015|
|           303676.79|2013|
|           298264.72|2014|
|           307539.97|2012|
|            293965.1|2016|
|           285010.22|2010|
|           276553.81|2011|
|           280317.58|2017|
+--------------------+----+



In [46]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
    SELECT ROUND(AVG(view), 2), price
    FROM homeSales
    WHERE price>=350000
    GROUP BY price
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+-------+
|round(avg(view), 2)|  price|
+-------------------+-------+
|               42.0| 444038|
|                5.0| 441664|
|                8.0| 402430|
|               41.5| 355163|
|                5.0| 447000|
|                1.0| 364447|
|               25.0| 429742|
|               17.0| 394217|
|               28.5| 389537|
|               34.0| 377555|
|               84.0|1402875|
|               27.0| 398212|
|               33.0| 373925|
|               20.0| 447682|
|               30.0| 446993|
|                5.0| 448678|
|               49.0| 444212|
|               36.0| 406122|
|               46.0| 448919|
|               41.0| 412586|
+-------------------+-------+
only showing top 20 rows

--- 2.0273959636688232 seconds ---


In [47]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table homeSales")

DataFrame[]

In [48]:
# 8. Check if the table is cached.
spark.catalog.isCached('homeSales')

True

In [49]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
    SELECT ROUND(AVG(view), 2), price
    FROM homeSales
    WHERE price>=350000
    GROUP BY price
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+-------------------+-------+
|round(avg(view), 2)|  price|
+-------------------+-------+
|               42.0| 444038|
|                5.0| 441664|
|                8.0| 402430|
|               41.5| 355163|
|                5.0| 447000|
|                1.0| 364447|
|               25.0| 429742|
|               17.0| 394217|
|               28.5| 389537|
|               34.0| 377555|
|               84.0|1402875|
|               27.0| 398212|
|               33.0| 373925|
|               20.0| 447682|
|               30.0| 446993|
|                5.0| 448678|
|               49.0| 444212|
|               36.0| 406122|
|               46.0| 448919|
|               41.0| 412586|
+-------------------+-------+
only showing top 20 rows

--- 0.8582882881164551 seconds ---


In [50]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales_df.write.parquet('parquet_homeSales_basic', mode='overwrite')

home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("homeSales_partitioned")

In [51]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('parquet_homeSales_basic')
p_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [52]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_homeSales')

In [53]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
    SELECT ROUND(AVG(view), 2), price
    FROM p_homeSales
    WHERE price>=350000
    GROUP BY price
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+-------+
|round(avg(view), 2)|  price|
+-------------------+-------+
|               42.0| 444038|
|                5.0| 441664|
|                8.0| 402430|
|               41.5| 355163|
|                5.0| 447000|
|                1.0| 364447|
|               25.0| 429742|
|               17.0| 394217|
|               28.5| 389537|
|               34.0| 377555|
|               84.0|1402875|
|               27.0| 398212|
|               33.0| 373925|
|               20.0| 447682|
|               30.0| 446993|
|                5.0| 448678|
|               49.0| 444212|
|               36.0| 406122|
|               46.0| 448919|
|               41.0| 412586|
+-------------------+-------+
only showing top 20 rows

--- 0.9840669631958008 seconds ---


In [54]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table homeSales")

DataFrame[]

In [55]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('homeSales')

False